# Dataset preprocessing

The goal of this notebook is to create a preprocessed kaggle dataset out of the competition dataset.  
For now, the preprocessing will be based on [this notebook](https://www.kaggle.com/code/vonmainstein/imu-tof).  
It consists of the following steps:
-   Set the appropriate dtypes (helps with RAM usage).
-   forward + backward filling feature columns
-   Converting the files csv files to parquet for faster loading and getting the correct data types upon loading.
-   Output statistics of the dataset into a csv(for readablity) file for standardization.  

> Note:  
> - The padding of the sequences will be performed in the model since we don't have access to the "leaderboard dataset" inputs.  
> - Demographics data set will be ignored for now.  

## Imports

In [48]:
import json
from os.path import join

import numpy as np
import pandas as pd
from pandas import DataFrame as DF
from kagglehub import whoami, competition_download, dataset_upload

from config import *

## Obtain raw dataset
Requires to be logged in if this notebook is not running on laggle, go to [your settings](https://www.kaggle.com/settings) to create an access token and put it in `~/.kaggle/`.

In [2]:
competition_dataset_path = competition_download(COMPETITION_HANDLE)

In [3]:
df = pd.read_csv(join(competition_dataset_path, "train.csv"), dtype=DATASET_DF_DTYPES)

In [4]:
features_describe = (
    df
    .drop(META_DATA_COLUMNS, axis="columns")
    .describe()
)
features_describe

,acc_x,acc_y,acc_z,rot_w,rot_x,rot_y,rot_z,thm_1,thm_2,thm_3,...,tof_5_v54,tof_5_v55,tof_5_v56,tof_5_v57,tof_5_v58,tof_5_v59,tof_5_v60,tof_5_v61,tof_5_v62,tof_5_v63
count,574945.000000,574945.000000,574945.000000,571253.000000,571253.000000,571253.000000,571253.000000,567958.000000,567307.000000,568473.000000,...,544803.000000,544803.000000,544803.000000,544803.000000,544803.000000,544803.000000,544803.000000,544803.000000,544803.000000,544803.000000
mean,1.639980,1.790704,-0.459811,0.360375,-0.119916,-0.059953,-0.188298,27.076448,27.133482,26.702993,...,29.395651,26.030826,45.342583,43.074842,40.045908,37.631707,34.977928,31.934330,29.024752,27.320358
std,5.781259,5.003945,6.096490,0.225739,0.465520,0.543028,0.504137,3.231948,2.941437,4.122353,...,58.093844,54.215523,68.466064,68.017631,66.941587,65.288710,63.201604,60.440645,57.218513,55.407192
min,-34.585938,-24.402344,-42.855469,0.000000,-0.999146,-0.999695,-0.998169,-0.370413,21.958820,0.000000,...,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000
25%,-2.964844,-2.121094,-5.417969,0.180237,-0.456299,-0.511536,-0.627686,24.753527,24.543737,24.640350,...,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000
50%,2.972656,0.695312,-1.562500,0.340332,-0.186890,-0.112610,-0.263916,26.982323,26.354338,26.956276,...,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000
75%,6.343750,6.816406,5.164062,0.503479,0.204590,0.440063,0.251099,29.425037,29.620148,29.231794,...,34.000000,24.000000,81.000000,76.000000,67.000000,59.000000,51.000000,42.000000,35.000000,31.000000
max,46.328125,27.183594,30.078125,0.999390,0.999817,0.999451,0.999878,38.457664,37.578339,37.294994,...,249.000000,249.000000,249.000000,249.000000,249.000000,249.000000,249.000000,249.000000,249.000000,249.000000


Impute -1 Time of Flight sensors values.

In [5]:
tof_cols = [col for col in df.columns if col.startswith("tof")]
to_replace = {col: -1.0 for col in df.columns if col.startswith("tof")}

df[tof_cols] = (
    df
    .loc[:, tof_cols]
     # df.replace with np.nan sets dtype to floar64 so we set it back to float32
    .replace(to_replace, value=np.nan)
    .astype("float32")
    .groupby(df["sequence_id"], observed=True, as_index=False)
    .ffill()
    .groupby(df["sequence_id"], observed=True, as_index=False)
    .bfill()
    # In case there are only nan in the column in the sequence
    .fillna(0)
)

One hot encode target values.

In [38]:
normed_sequence_len = int(df["sequence_counter"].quantile(SEQUENCE_NORMED_LEN_QUANTILE))
sequence_it = df.groupby("sequence_id", observed=True, as_index=False)
x = np.empty(
    shape=(len(sequence_it), normed_sequence_len, df.shape[1] - len(META_DATA_COLUMNS)),
    dtype="float32"
)

def normed_feature_sequence_len(sequence: np.ndarray, normed_sequence_len: int) -> np.ndarray:
    features = (
        sequence
        .drop(columns=META_DATA_COLUMNS)
        .values
    )
    len_diff = abs(normed_sequence_len - len(features))
    if len(features) < normed_sequence_len:
        padded_features = np.pad(
            features,
            ((len_diff // 2 + len_diff % 2, len_diff // 2), (0, 0)),
        )
        return padded_features
    elif len(features) > normed_sequence_len:
        truncated_features = features[len_diff // 2:-len_diff // 2]
        return truncated_features
    else:
        return features

for sequence_idx, (sequence_id, sequence) in enumerate(sequence_it):
    x[sequence_idx] = normed_feature_sequence_len(sequence, normed_sequence_len)

In [47]:
one_hot_encoded_targets = pd.get_dummies(df["gesture"], dtype="float32")
one_hot_encoded_targets.columns.to_list()

['Above ear - pull hair',
 'Cheek - pinch skin',
 'Eyebrow - pull hair',
 'Eyelash - pull hair',
 'Feel around in tray and pull out an object',
 'Forehead - pull hairline',
 'Forehead - scratch',
 'Neck - pinch skin',
 'Neck - scratch',
 'Text on phone',
 'Wave hello',
 'Write name in air',
 'Write name on leg',
 'Drink from bottle/cup',
 'Pinch knee/leg skin',
 'Pull air toward your face',
 'Scratch knee/leg skin',
 'Glasses on/off']

In [49]:
# Create dataset direcory
! mkdir -p preprocessed_dataset
! rm preprocessed_dataset/*
# Save dataframes
np.save("preprocessed_dataset/X.npy", x, allow_pickle=False)
features_describe.to_csv("preprocessed_dataset/features_describe.csv")
one_hot_encoded_targets = pd.get_dummies(df["gesture"], dtype="float32")
np.save("preprocessed_dataset/Y.npy", one_hot_encoded_targets.values, allow_pickle=False)
with open("preprocessed_dataset/target_names_list.json", "w") as fp:
    json.dump(one_hot_encoded_targets.columns.to_list(), fp, indent=1)

In [50]:
if input("Do you want to upload the  dataset to kaggle?[yes/no]") == "yes":
    # Updaload the dataset
    dataset_upload(
        join(whoami()["username"], "prepocessed-cmi-2025"),
        "preprocessed_dataset",
        version_notes="Preprocessed Child Mind Institue 2025 competition dataset."
    )

Kaggle credentials successfully validated.
Uploading Dataset https://www.kaggle.com/datasets/mauroabidalcarrer/prepocessed-cmi-2025 ...
Starting upload for file preprocessed_dataset/target_names_list.json


Uploading: 100%|██████████| 441/441 [00:00<00:00, 726B/s]

Upload successful: preprocessed_dataset/target_names_list.json (441B)
Starting upload for file preprocessed_dataset/features_describe.csv



Uploading: 100%|██████████| 27.6k/27.6k [00:00<00:00, 68.2kB/s]

Upload successful: preprocessed_dataset/features_describe.csv (27KB)
Starting upload for file preprocessed_dataset/Y.npy



Uploading: 100%|██████████| 41.4M/41.4M [00:04<00:00, 9.22MB/s]

Upload successful: preprocessed_dataset/Y.npy (39MB)
Starting upload for file preprocessed_dataset/X.npy



Uploading: 100%|██████████| 1.23G/1.23G [01:48<00:00, 11.3MB/s]

Upload successful: preprocessed_dataset/X.npy (1GB)


Your dataset has been created.
Files are being processed...
See at: https://www.kaggle.com/datasets/mauroabidalcarrer/prepocessed-cmi-2025
